<a href="https://colab.research.google.com/github/dinaldoap/jit-sdp-nn/blob/master/notebook/mlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data
import random
from scipy.stats import mstats

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/dinaldoap/jit-sdp-data/master/jenkins.csv')
df.head()

,fix,ns,nd,nf,entropy,la,ld,lt,ndev,age,nuc,exp,rexp,sexp,author_date_unix_timestamp,classification,contains_bug
0,False,7.0,7.0,7.0,2.641604,9.0,9.0,426.428571,100.0,0.000093,1.0,5171.0,30.227271,1472.714286,1555326371,None,False
1,False,7.0,7.0,7.0,2.750000,8.0,8.0,426.428571,100.0,6.314775,2.0,5170.0,29.227271,1471.714286,1555326363,None,False
2,False,1.0,1.0,2.0,0.906580,15.0,44.0,96.000000,4.0,0.034722,2.0,629.0,14.828373,414.000000,1554971763,None,False
3,False,1.0,1.0,1.0,0.000000,0.0,0.0,40.000000,1.0,0.000012,1.0,4.0,3.058824,3.000000,1554969774,None,False
4,False,1.0,2.0,4.0,1.662506,14.0,10.0,67.000000,6.0,21.280683,4.0,3.0,2.058824,2.000000,1554967752,Feature Addition,False


In [3]:
df = df.sample(frac=1)
label_col = 'contains_bug'
features_cols = ['fix', 'ns', 'nd', 'nf', 'entropy', 'la', 'ld', 'lt', 'ndev', 'age', 'nuc', 'exp', 'rexp', 'sexp', 'classification']
x = df[features_cols]
x['fix'] = x['fix'].astype('int')
df_classification = pd.get_dummies(x, columns=['classification'])
x = pd.concat([x, df_classification], axis='columns')
x = x.drop(['classification'], axis='columns')
x = x.values
y = df[label_col]
y = y.astype('category')
y = y.cat.codes
y = y.values

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [4]:
print(x[:3])
print(y[:3])

[[0.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
  0.00000000e+00 3.00000000e+00 0.00000000e+00 2.79000000e+02
  1.00000000e+01 1.97685185e-01 1.00000000e+00 1.39000000e+02
  6.40666667e+01 8.20000000e+01 0.00000000e+00 1.00000000e+00
  1.00000000e+00 1.00000000e+00 0.00000000e+00 3.00000000e+00
  0.00000000e+00 2.79000000e+02 1.00000000e+01 1.97685185e-01
  1.00000000e+00 1.39000000e+02 6.40666667e+01 8.20000000e+01
  0.00000000e+00 0.00000000e+00 0.00000000e+00 1.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
  0.00000000e+00 4.00000000e+00 0.00000000e+00 1.98000000e+02
  7.00000000e+00 3.75470394e+02 1.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 1.00000000e+00
  1.00000000e+00 1.00000000e+00 0.00000000e+00 4.00000000e+00
  0.00000000e+00 1.98000000e+02 7.00000000e+00 3.75470394e+02
  1.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 1.00000000e+00 0.000

In [0]:
val_index = int( len(x) * 0.8 )
test_index = int( len(x) * 0.9 )
x_train, y_train = x[:val_index], y[:val_index]
x_val, y_val = x[val_index:test_index], y[val_index:test_index]
x_test, y_test = x[test_index:], y[test_index:]

In [6]:
print(x_train[:3])
print(x_val[:3])
print(x_test[:3])

[[0.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
  0.00000000e+00 3.00000000e+00 0.00000000e+00 2.79000000e+02
  1.00000000e+01 1.97685185e-01 1.00000000e+00 1.39000000e+02
  6.40666667e+01 8.20000000e+01 0.00000000e+00 1.00000000e+00
  1.00000000e+00 1.00000000e+00 0.00000000e+00 3.00000000e+00
  0.00000000e+00 2.79000000e+02 1.00000000e+01 1.97685185e-01
  1.00000000e+00 1.39000000e+02 6.40666667e+01 8.20000000e+01
  0.00000000e+00 0.00000000e+00 0.00000000e+00 1.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
  0.00000000e+00 4.00000000e+00 0.00000000e+00 1.98000000e+02
  7.00000000e+00 3.75470394e+02 1.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 1.00000000e+00
  1.00000000e+00 1.00000000e+00 0.00000000e+00 4.00000000e+00
  0.00000000e+00 1.98000000e+02 7.00000000e+00 3.75470394e+02
  1.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 1.00000000e+00 0.000

In [0]:
mean_train = x_train.mean(axis=0)
std_train = x_train.std(axis=0)

In [8]:
print(mean_train)
print(std_train)

[2.77010603e-01 1.59001810e+00 2.51574864e+00 3.99772433e+00
 7.30469272e-01 5.45791052e+01 2.81344712e+01 7.08145326e+02
 1.90090509e+01 8.36493486e+01 1.93943626e+00 1.74181190e+03
 8.56089896e+01 9.00974561e+02 2.77010603e-01 1.59001810e+00
 2.51574864e+00 3.99772433e+00 7.30469272e-01 5.45791052e+01
 2.81344712e+01 7.08145326e+02 1.90090509e+01 8.36493486e+01
 1.93943626e+00 1.74181190e+03 8.56089896e+01 9.00974561e+02
 2.77010603e-01 1.59296612e-01 3.52728213e-02 4.76027929e-01
 1.58262219e-02 3.65658133e-02]
[4.47521763e-01 1.61578510e+00 6.37581314e+00 2.48750879e+01
 1.11787668e+00 7.46209584e+02 6.18922690e+02 1.06163544e+03
 2.68574602e+01 2.18006004e+02 7.51380104e+00 1.97913371e+03
 8.21701552e+01 1.30899047e+03 4.47521763e-01 1.61578510e+00
 6.37581314e+00 2.48750879e+01 1.11787668e+00 7.46209584e+02
 6.18922690e+02 1.06163544e+03 2.68574602e+01 2.18006004e+02
 7.51380104e+00 1.97913371e+03 8.21701552e+01 1.30899047e+03
 4.47521763e-01 3.65952458e-01 1.84468559e-01 4.99425

In [0]:
x_train = (x_train - mean_train) / std_train
x_val = (x_val - mean_train) / std_train
x_test = (x_test - mean_train) / std_train

In [10]:
print(x_train[:3])
print(x_val[:3])
print(x_test[:3])

[[-0.618988   -0.36515877 -0.23773417 -0.12051111 -0.65344352 -0.06912147
  -0.04545717 -0.40423041 -0.33543942 -0.38279525 -0.1250281  -0.80985528
  -0.26216724 -0.62565357 -0.618988   -0.36515877 -0.23773417 -0.12051111
  -0.65344352 -0.06912147 -0.04545717 -0.40423041 -0.33543942 -0.38279525
  -0.1250281  -0.80985528 -0.26216724 -0.62565357 -0.618988   -0.43529319
  -0.19121319  1.04915065 -0.12680977 -0.19481688]
 [-0.618988   -0.36515877 -0.23773417 -0.12051111 -0.65344352 -0.06778137
  -0.04545717 -0.48052778 -0.44714023  1.33859178 -0.1250281  -0.88008803
  -1.04185016 -0.68829727 -0.618988   -0.36515877 -0.23773417 -0.12051111
  -0.65344352 -0.06778137 -0.04545717 -0.48052778 -0.44714023  1.33859178
  -0.1250281  -0.88008803 -1.04185016 -0.68829727 -0.618988    2.2973022
  -0.19121319 -0.95315196 -0.12680977 -0.19481688]
 [-0.618988   -0.36515877 -0.23773417 -0.12051111 -0.65344352 -0.06108084
  -0.04545717  0.10912849  0.14859741 -0.37918642 -0.1250281  -0.16361295
   0.426414

In [0]:
def create_sampler(y):
  _, counts = np.unique(y, return_counts=True)
  n_samples = len(y)
  class_weights = n_samples / counts
  weights = class_weights[y]
  return data.WeightedRandomSampler(weights=weights, num_samples=n_samples, replacement=True)

In [0]:
sampler = create_sampler(y_train)

In [0]:
x_train, y_train = torch.from_numpy(x_train), torch.from_numpy(y_train)
x_val, y_val = torch.from_numpy(x_val), torch.from_numpy(y_val)
x_test, y_test = torch.from_numpy(x_test), torch.from_numpy(y_test)

In [0]:
train_dataset = data.TensorDataset(x_train, y_train)
val_dataset = data.TensorDataset(x_val, y_val)
test_dataset = data.TensorDataset(x_test, y_test)

train_dataloader = data.DataLoader(train_dataset, batch_size=512, sampler=sampler)
val_dataloader = data.DataLoader(val_dataset, batch_size=32)
test_dataloader = data.DataLoader(test_dataset, batch_size=32)

In [15]:
inputs, targets = next(iter(train_dataloader))
print(inputs[:3])
print(targets[:3])

inputs, targets = next(iter(val_dataloader))
print(inputs[:3])
print(targets[:3])

inputs, targets = next(iter(test_dataloader))
print(inputs[:3])
print(targets[:3])

tensor([[-6.1899e-01, -3.6516e-01,  7.5951e-02,  9.1484e-05,  9.6358e-01,
         -3.1599e-02, -3.8994e-02, -2.4645e-01, -5.9607e-01, -2.5859e-01,
          2.7424e-01,  5.7156e-01,  1.5072e+00,  1.2185e+00, -6.1899e-01,
         -3.6516e-01,  7.5951e-02,  9.1484e-05,  9.6358e-01, -3.1599e-02,
         -3.8994e-02, -2.4645e-01, -5.9607e-01, -2.5859e-01,  2.7424e-01,
          5.7156e-01,  1.5072e+00,  1.2185e+00, -6.1899e-01, -4.3529e-01,
         -1.9121e-01,  1.0492e+00, -1.2681e-01, -1.9482e-01],
        [-6.1899e-01, -3.6516e-01, -2.3773e-01, -1.2051e-01, -6.5344e-01,
         -2.4898e-02, -4.5457e-02, -2.7707e-01, -6.3331e-01, -3.6440e-01,
         -1.2503e-01,  6.5088e-01,  1.7223e+00,  1.3308e+00, -6.1899e-01,
         -3.6516e-01, -2.3773e-01, -1.2051e-01, -6.5344e-01, -2.4898e-02,
         -4.5457e-02, -2.7707e-01, -6.3331e-01, -3.6440e-01, -1.2503e-01,
          6.5088e-01,  1.7223e+00,  1.3308e+00, -6.1899e-01,  2.2973e+00,
         -1.9121e-01, -9.5315e-01, -1.2681e-01, -1

In [0]:
class Classifier(nn.Module):
  def __init__(self, epochs, input_size, hidden_size, drop_prob, val_loss=None):
    super(Classifier, self).__init__()
    self.epochs = epochs
    self.input_size = input_size
    self.hidden_size = hidden_size
    self.drop_prob = drop_prob
    self.val_loss = val_loss
    self.fc1 = nn.Linear(input_size, hidden_size)
    self.fc2 = nn.Linear(hidden_size, hidden_size)
    self.fcout = nn.Linear(hidden_size, 1)
    self.dropout = nn.Dropout(drop_prob)
  
  def forward(self, x):
    x = torch.relu(self.fc1(x))
    x = self.dropout(x)
    x = torch.relu(self.fc2(x))
    x = self.dropout(x)
    x = torch.sigmoid(self.fcout(x))
    return x

In [17]:
classifier = Classifier(epochs=200, input_size=x.shape[1], hidden_size=128, drop_prob=0.2)
classifier

Classifier(
  (fc1): Linear(in_features=34, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=128, bias=True)
  (fcout): Linear(in_features=128, out_features=1, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)

In [0]:
filename = 'classifier.cpt'
def save_classifier(classifier):
  checkpoint = {
      'epochs': classifier.epochs,
      'input_size': classifier.input_size,
      'hidden_size': classifier.hidden_size,
      'drop_prob': classifier.drop_prob,
      'val_loss': classifier.val_loss,
      'state_dict': classifier.state_dict()
  }
  with open(filename, 'wb') as f:
    torch.save(checkpoint, f)

def load_classifier():
  with open(filename, 'rb') as f:
    checkpoint = torch.load(f)
  classifier = Classifier(checkpoint['epochs'], checkpoint['input_size'], checkpoint['hidden_size'], checkpoint['drop_prob'], checkpoint['val_loss'])
  classifier.load_state_dict(checkpoint['state_dict'])
  return classifier

In [0]:
save_classifier(classifier)
classifier = load_classifier()

In [20]:
if torch.cuda.is_available():
  classifier = classifier.cuda()
  inputs = inputs.cuda()

classifier(inputs[:3].float())

tensor([[0.4887],
        [0.5433],
        [0.5303]], device='cuda:0', grad_fn=<SigmoidBackward>)

In [0]:
criterion = nn.BCELoss()
optimizer = optim.Adam(params=classifier.parameters(), lr=0.003)

In [22]:
train_loss = 0
epochs = classifier.epochs
for epoch in range(epochs):
  classifier.train()
  for inputs, targets in train_dataloader:
    if torch.cuda.is_available():
      inputs, targets = inputs.cuda(), targets.cuda()

    outputs = classifier(inputs.float())
    loss = criterion(outputs.squeeze(), targets.float())
    train_loss += loss.item()

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
  
  with torch.no_grad():
    classifier.eval()
    val_loss = 0
    for inputs, targets in val_dataloader:
      if torch.cuda.is_available():
        inputs, targets = inputs.cuda(), targets.cuda()

      outputs = classifier(inputs.float())
      loss = criterion(outputs.squeeze(), targets.float())
      val_loss += loss.item()

    train_loss = train_loss / len(train_dataloader)
    val_loss = val_loss / len(val_dataloader)
    print('Epoch: {}, Train loss: {}, Val loss: {}'.format(epoch, train_loss, val_loss))

    if classifier.val_loss is None or val_loss < classifier.val_loss:
      classifier.val_loss = val_loss
      classifier.epochs = epoch
      save_classifier(classifier)

Epoch: 0, Train loss: 0.6078970902844479, Val loss: 0.5394719101880726
Epoch: 1, Train loss: 0.5697627487770408, Val loss: 0.5291634845106226
Epoch: 2, Train loss: 0.555106465291014, Val loss: 0.4810828637135656
Epoch: 3, Train loss: 0.5395144068429092, Val loss: 0.5141895338892937
Epoch: 4, Train loss: 0.5315664680987837, Val loss: 0.46009924929392965
Epoch: 5, Train loss: 0.5329518560573465, Val loss: 0.525476542350493
Epoch: 6, Train loss: 0.5226541716906192, Val loss: 0.5118679482685892
Epoch: 7, Train loss: 0.5124962038613535, Val loss: 0.4584381843083783
Epoch: 8, Train loss: 0.5030729410073662, Val loss: 0.5071689517874467
Epoch: 9, Train loss: 0.5050930465444701, Val loss: 0.47921101239166763
Epoch: 10, Train loss: 0.508087327828493, Val loss: 0.4886385880802807
Epoch: 11, Train loss: 0.5009951243994133, Val loss: 0.4895663724133843
Epoch: 12, Train loss: 0.4939860308123197, Val loss: 0.5244554865517115
Epoch: 13, Train loss: 0.49797044355605924, Val loss: 0.489129833092815
Epo

In [0]:
def calculate_recalls(targets, predictions):
  n_classes = 2
  confusion_matrix, _, _ = np.histogram2d(targets.detach().cpu().numpy(), predictions.squeeze().detach().cpu().numpy(), bins=n_classes)
  return np.diag(confusion_matrix) / (np.sum(confusion_matrix, axis=1))

def gmean(classifier, dataloader):
  with torch.no_grad():
    classifier.eval()
    recalls = np.zeros((2))
    for inputs, targets in dataloader:
      if torch.cuda.is_available():
        inputs, targets = inputs.cuda(), targets.cuda()
      
      outputs = classifier(inputs.float())
      predictions = torch.round(outputs).int()
      recalls += calculate_recalls(targets, predictions)

    recalls = recalls / len(dataloader)
    return mstats.gmean(recalls)    

In [0]:
def evaluate(classifier):
  if torch.cuda.is_available():
    classifier = classifier.cuda()
  train_accuracy = gmean(classifier, train_dataloader)
  test_accuracy = gmean(classifier, test_dataloader)
  print('Epochs: {}'.format(classifier.epochs))
  print('Train g-mean: {}'.format(train_accuracy.item()))
  print('Test g-mean: {}'.format(test_accuracy.item()))

In [25]:
print('Best classifier')
evaluate(load_classifier())

Best classifier
Epochs: 81
Train g-mean: 0.7964218100449051
Test g-mean: 0.731752076399906
